In [178]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
import requests
import numpy as np
import pandas as pd
import re
from pymongo import MongoClient

In [180]:
def pars_hh(link, result, min_salary):
    headers = {'User-Agent':
               'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
    req = requests.get(link, headers=headers)
    html = bs(req.text, 'lxml')
    vacancy_list = html.findAll('div', {'data-qa':['vacancy-serp__vacancy vacancy-serp__vacancy_premium', 'vacancy-serp__vacancy']})
    for vacancy in vacancy_list:
        Dict_temp = {}
        info = vacancy.find('div', {'class':'resume-search-item__name'}).findChild().findChild()
        name = info.getText()
        link = info['href']
        compensation = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
        if compensation:
            spl_comp = compensation.split(' ')
            if 'от' in spl_comp:
                spl_comp.remove('от')
            if 'до' in spl_comp:
                spl_comp.remove('до')
            currency = spl_comp[1]
            real_compensation = spl_comp[0]
            if '-' in real_compensation:
                min_comp = real_compensation.split('-')[0]
                max_comp = real_compensation.split('-')[1]
            else:
                min_comp = max_comp = real_compensation
        else:
            continue
        regex = re.compile('\s+')
        max_comp = int(regex.sub('', max_comp))
        min_comp = int(regex.sub('', min_comp))
        if max_comp < min_salary:
            continue
        Dict_temp['name'] = name
        Dict_temp['min'] = min_comp
        Dict_temp['max'] = max_comp
        Dict_temp['currency'] = currency
        Dict_temp['link'] = link
        Dict_temp['site'] = 'superjob.ru'
        result.append(Dict_temp)
        
    return result

def pars_sj(link, result, min_salary):
    headers = {'User-Agent':
               'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
    req = requests.get(link_sj, headers=headers)
    html = bs(req.text, 'lxml')
    vacancy_list = html.findAll('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})
    for vacancy in vacancy_list:
        Dict_temp = {}
        info = vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).getText()
        salary = vacancy.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText()
        if not salary or 'По' in salary:
            continue
        else:
            if 'от' in salary:
                salary = salary.replace('от', '')
            if 'до' in salary:
                salary = salary.replace('до', '')
            currency = salary[-1]
            salary = salary[:-1]
            if '—' in salary:
                min_comp = salary.split('—')[0]
                max_comp = salary.split('—')[1]
            else:
                min_comp = salary
                max_comp = salary
        regex = re.compile('\s+')
        max_comp = int(regex.sub('', max_comp))
        min_comp = int(regex.sub('', min_comp))
        if max_comp < min_salary:
            continue
        link = main_link_sj + vacancy.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).findParent()['href']
        Dict_temp['name'] = info
        Dict_temp['min'] = min_comp
        Dict_temp['max'] = max_comp
        Dict_temp['currency'] = currency
        Dict_temp['link'] = link
        Dict_temp['site'] = 'superjob.ru'
        result.append(Dict_temp)
        
    return result

In [143]:
main_link_sj = 'https://www.superjob.ru'
spec_link_sj = '/vacancy/search/?keywords='+spec
link_sj = main_link_sj + spec_link_sj
headers = {'User-Agent':
               'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
req = requests.get(link_sj, headers=headers)
html = bs(req.text, 'lxml')
vacancy_list = html.findAll('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})


In [181]:
result = []
n = int(input('Сколько страниц парсить?\t'))
main_link = 'https://hh.ru/search/vacancy'
spec = input('Введите должность\t')
spec_link = '?text=' + spec
link = main_link + spec_link
main_link_sj = 'https://www.superjob.ru'
spec_link_sj = '/vacancy/search/?keywords='+spec
link_sj = main_link_sj + spec_link_sj
min_salary = int(input('Минимальная зарплата?\t'))
i = 1
while i <= n:
    result = pars_hh(link, result, min_salary)
    result = pars_sj(link_sj, result, min_salary)
    link += '&page='+str(i)
    link_sj += '&page='+str(i+1)
    i += 1  
result
client = MongoClient('localhost', 27017)
db = client['BD1']
vacancy = db.vacancy
for res in result:
    vacancy.insert_one(res)

Сколько страниц парсить?	3
Введите должность	python
Минимальная зарплата?	50000


[{'name': 'Ведущий разработчик Python',
  'min': 160000,
  'max': 160000,
  'currency': 'руб.',
  'link': 'https://ulyanovsk.hh.ru/vacancy/34200113?query=python',
  'site': 'superjob.ru'},
 {'name': 'Тестировщик ПО',
  'min': 50000,
  'max': 65000,
  'currency': 'руб.',
  'link': 'https://ulyanovsk.hh.ru/vacancy/34317091?query=python',
  'site': 'superjob.ru'},
 {'name': 'QA Automation Engineer',
  'min': 120000,
  'max': 150000,
  'currency': 'руб.',
  'link': 'https://ulyanovsk.hh.ru/vacancy/32370826?query=python',
  'site': 'superjob.ru'},
 {'name': 'QA Automation Engineer',
  'min': 120000,
  'max': 150000,
  'currency': 'руб.',
  'link': 'https://ulyanovsk.hh.ru/vacancy/32370885?query=python',
  'site': 'superjob.ru'},
 {'name': 'QA Automation Engineer',
  'min': 120000,
  'max': 150000,
  'currency': 'руб.',
  'link': 'https://ulyanovsk.hh.ru/vacancy/32370952?query=python',
  'site': 'superjob.ru'},
 {'name': 'Full-stack developer',
  'min': 120000,
  'max': 120000,
  'currency':